<a href="https://colab.research.google.com/github/alinnman/lineofsight/blob/main/mallorca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple sample for line of sight
This sample calculates the line of sight from a target (lighthouse/mountain/skyscraper) to two observers located at various heights over the sea surface. All **heights** are specified in meters. Calculations are made without and with refraction. For refraction **air pressure** (kPa), **temperature** (degrees Celsius) and **temperature gradient** (degrees Celsius / meter) must be entered. Default values are pre-set.

A calculation for an observer at sea level is also made, and a check is made to see if the parabola approximation ("8 inches per miles squared") is correct or not.

A calculation of obstructed height is also made, and presented as a separate table.

Instructions for use: Fill in the parameters and **Press Ctrl-F9 or the arrow button below** to calculate the result which will be presented in the bottom cell.

*The pre-set values are taken from [the view](https://maps.app.goo.gl/QEKNpBXzX8unTxtp7) of Puig Major (1436) on Mallorca Island from Penyagolosa (1813 m) on the Spanish mainland.*

Allow some time for the first execution (Google Colab needs some time to start up).<br/>
When running for the first time you will get a warning about the code not originating from Google Colab (it is stored in GitHub). You can safely ignore this warning, but you may of course read the source code. Click "Show code" to inspect the source code (you may also modify).

*Tip: To simulate severe refraction set the temperature gradient to 0.1 (i.e strong temperature inversion)*.

In [4]:
from math import sqrt, pi, atan2, cos
from IPython.display import Markdown, display

# @markdown ### Enter parameters:
TARGET_HEIGHT = 1436 # @param {"type":"integer"}
TARGET_DISTANCE = 272000 # @param {"type":"integer"}
OBSERVER_HEIGHT_1 = 1000 # @param {"type":"integer"}
OBSERVER_HEIGHT_2 = 1813 # @param {"type":"integer"}
PRESSURE = 101 # @param {"type":"integer"}
TEMPERATURE = 20 # @param {"type":"integer"}
TEMP_GRADIENT = -0.01 # @param {"type":"number"}

EARTH_CIRCUMFERENCE_EQUATORIAL = 40075.017
EARTH_CIRCUMFERENCE_MERIDIONAL = 40007.86
EARTH_CIRCUMFERENCE = (EARTH_CIRCUMFERENCE_EQUATORIAL + EARTH_CIRCUMFERENCE_MERIDIONAL) / 2
EARTH_RADIUS = EARTH_CIRCUMFERENCE / (2 * pi)

R = EARTH_RADIUS*1000

def visibility_limit (RR : float, h1 : float, h2 : float,\
                      get_obscurations : bool = False,
                      target_distance = None)\
    -> tuple [float, float, list[tuple[float,float]]]:
  ''' Geometry for line-of-sight '''
  x1a = sqrt (((RR + h1)**2) - RR**2)
  x1r = atan2 (x1a, RR)
  x1 = x1r * RR
  x2a = sqrt (((RR + h2)**2) - RR**2)
  x2r = atan2 (x2a, RR)
  x2 = x2r * RR

  obscurations = list()
  output_target_done = False

  # Do split and get obscured height
  if get_obscurations:
    for i in range (10 + 1):
      s = x1r * ((10 - i) / 10)
      ho = RR / cos (s) - RR
      dist = x2 + s*RR
      if dist < target_distance and not output_target_done:
        dist2 = target_distance - x2
        angle = dist2 / RR
        hoPrim = RR / cos (angle) - RR
        obscurations.append ((target_distance, hoPrim, True))
        output_target_done = True
      obscurations.append ((dist, ho, False))
    # obscurations.append ()

  return x1, x2, obscurations

def reportObsc (obsc : list[tuple[float,float, bool]]) :
  display (Markdown ("Obscured heights (with refraction)"))
  tableString = "| Distance (m) | Obstructed Height (m) | Visible part (%) |\n"
  tableString += "| ----- | ----- | ----- |\n"

  for e in obsc:
    distance = e[0]
    distanceString = str(round(distance))
    obscHeight = e[1]
    obscTarget = e[2]

    if obscHeight == 0 and distance > 0:
      distanceString = distanceString + " and closer"
    fattyMarker = ""
    if obscTarget:
      fattyMarker = "**"
    percent = 100 - (obscHeight / TARGET_HEIGHT) * 100
    if percent < 0:
      percent = 0.0
    elif percent > 100:
      percent = 100.0
    tableString += "|" + fattyMarker + distanceString + fattyMarker +\
                   "|" + fattyMarker + str(round(obscHeight,2)) + fattyMarker + \
                   "|" + fattyMarker + str(round(percent,2)) + fattyMarker + "|\n"
  display (Markdown(tableString))

DT_DH = TEMP_GRADIENT # Temperature shift with increasing elevation
K_FACTOR = 503*(PRESSURE*10)*(1/((TEMPERATURE+273)**2))*(0.0343 + DT_DH)
Ra = R / (1 - K_FACTOR) # This is the radius adjusted for refraction
# See https://en.wikipedia.org/wiki/Atmospheric_refraction#cite_note-Hirt2010-28

h1 = TARGET_HEIGHT # Height of target

display (Markdown ("# Results"))
display (Markdown ("HEIGHT of TARGET = " + str(h1) + " meters"))

for h2 in [0, OBSERVER_HEIGHT_1, OBSERVER_HEIGHT_2]: # Try some different observer elevations

  display (Markdown ("### CALCULATING for OBSERVER HEIGHT = " + str(h2) + " meters"))

  ## No refraction

  x1, x2, obsc = visibility_limit (R, h1, h2)
  distance = x1 + x2
  lightHouseTable = "|Refraction|Line of Sight (meters)|\n"
  lightHouseTable += "|-----|-----|\n"
  lightHouseTable += "|No|" + str(round(distance))+ "|\n"

  # For zero elevation (observer at sea level) we can double-check with the parabola approximation
  # NOTE: The parabola approximation is useless if observer is above sea level!
  if h2 == 0:
    outputStr = "*Checking the 8 inches per miles squared approximation*<br/>"
    MILES_PER_KM = 0.621371
    miles = (distance/1000) * MILES_PER_KM
    inches = 8 * (miles**2)
    INCHES_PER_METER = 39.3701
    meters_from_inches = inches / INCHES_PER_METER
    outputStr += "*Parabola approximation of lighthouse height = " + str(round(meters_from_inches,5)) + " meters*<br/>"
    outputStr += "*.. which is " + \
                 str(abs(round(((meters_from_inches - h1) / h1),6)*100)) + \
                 " % off the real value*"
    display (Markdown (outputStr))

  ## Now adjust for refraction

  x1,x2,obsc = visibility_limit (Ra, h1, h2, get_obscurations=True, target_distance=TARGET_DISTANCE)
  distance = x1 + x2

  lightHouseTable += "|Yes|" + str(round(distance))+ "|\n"
  display (Markdown(lightHouseTable))
  if obsc is not None:
    reportObsc (obsc)

# Results

HEIGHT of TARGET = 1436 meters

### CALCULATING for OBSERVER HEIGHT = 0 meters

*Checking the 8 inches per miles squared approximation*<br/>*Parabola approximation of lighthouse height = 1435.68157 meters*<br/>*.. which is 0.0222 % off the real value*

|Refraction|Line of Sight (meters)|
|-----|-----|
|No|135275|
|Yes|146196|


...

| Distance (m) | Obstructed Height (m) | Visible part (%) |
| ----- | ----- | ----- |
|**272000**|**1511.67**|**0.0**|
|268198|1436.0|0.0|
|253578|1163.12|19.0|
|238959|918.99|36.0|
|224339|703.58|51.0|
|209720|516.91|64.0|
|195100|358.96|75.0|
|180481|229.73|84.0|
|165861|129.22|91.0|
|151241|57.43|96.0|
|136622|14.36|99.0|
|122002 and closer|0.0|100.0|


### CALCULATING for OBSERVER HEIGHT = 1813 meters

|Refraction|Line of Sight (meters)|
|-----|-----|
|No|287269|
|Yes|310462|


Obscured heights (with refraction)

| Distance (m) | Obstructed Height (m) | Visible part (%) |
| ----- | ----- | ----- |
|310462|1436.0|0.0|
|295842|1163.12|19.0|
|281222|918.99|36.0|
|**272000**|**779.76**|**45.7**|
|266603|703.58|51.0|
|251983|516.91|64.0|
|237364|358.96|75.0|
|222744|229.73|84.0|
|208125|129.22|91.0|
|193505|57.43|96.0|
|178885|14.36|99.0|
|164266 and closer|0.0|100.0|
